In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import optuna
import os
import json
import copy
import joblib
import pandas as pd
import numpy as np
import time
from typing import Dict, Any, Optional, Union, List, Tuple, Type

from constants import DEVICE
from data_handling import DataHandler
from constants import RESULTS_DIR, MODELS_DIR, PREDS_DIR
from metrics import weighted_cross_entropy_loss
from mlp_model import *

Using MPS device (Apple Silicon GPU)


In [2]:
dh = DataHandler()

DataHandler initialized - Using 114 features - Test year: 2020


In [3]:
# First, set up the ASHA pruner
asha_pruner = optuna.pruners.SuccessiveHalvingPruner(
    min_resource=20,        # Minimum number of steps before pruning
    reduction_factor=2,    # Reduction factor for successive halving
    min_early_stopping_rate=0
)

study_mlp0 = optuna.create_study(study_name='mlp0', direction="minimize", pruner=asha_pruner)
study_mlp0.optimize(
    lambda trial: objective_mlp(trial,
                            input_dim=dh.input_dim,
                            depth=0,
                            dataloaders=dh.get_nn_data('cv', batch_size=256),
                            max_epochs=80),
    n_trials=20,  # Number of trials to run
    timeout=900,   # Timeout in 15 mins
    n_jobs=-1,     # Use all available cores
)

# Save the complete study to a file
study_filename = f"mlp_studies/mlp0_optuna_study_{time.strftime('%Y%m%d_%H%M%S')}.pkl"
joblib.dump(study_mlp0, study_filename)
print(f"Study saved to {study_filename}")

[I 2025-05-11 19:25:47,232] A new study created in memory with name: mlp0
[I 2025-05-11 19:26:24,169] Trial 5 pruned. 
[I 2025-05-11 19:26:25,656] Trial 7 pruned. 
[I 2025-05-11 19:26:27,321] Trial 3 pruned. 
[I 2025-05-11 19:26:31,557] Trial 1 pruned. 
[I 2025-05-11 19:27:00,431] Trial 8 pruned. 
[I 2025-05-11 19:27:04,853] Trial 9 pruned. 
[I 2025-05-11 19:27:06,656] Trial 10 pruned. 
[I 2025-05-11 19:27:07,939] Trial 0 pruned. 
[I 2025-05-11 19:27:13,105] Trial 11 pruned. 
[I 2025-05-11 19:27:13,916] Trial 2 pruned. 
[I 2025-05-11 19:27:37,374] Trial 12 pruned. 
[I 2025-05-11 19:27:46,546] Trial 14 pruned. 
[I 2025-05-11 19:28:03,990] Trial 6 finished with value: 1.0438262010231996 and parameters: {'learning_rate': 0.002423032894832007, 'weight_decay': 0.00012880192230710512}. Best is trial 6 with value: 1.0438262010231996.
[I 2025-05-11 19:28:09,086] Trial 4 finished with value: 1.0438333107874944 and parameters: {'learning_rate': 0.0023541807073186862, 'weight_decay': 0.0016227249

Study saved to mlp_studies/mlp0_optuna_study_20250511_192856.pkl


In [4]:
# First, set up the ASHA pruner
asha_pruner = optuna.pruners.SuccessiveHalvingPruner(
    min_resource=20,        # Minimum number of steps before pruning
    reduction_factor=2,    # Reduction factor for successive halving
    min_early_stopping_rate=0
)

study_mlp1 = optuna.create_study(study_name='mlp1', direction="minimize", pruner=asha_pruner)
study_mlp1.optimize(
    lambda trial: objective_mlp(trial,
                            input_dim=dh.input_dim,
                            depth=1,
                            dataloaders=dh.get_nn_data('cv', batch_size=256),
                            max_epochs=120),
    n_trials=40,  # Number of trials to run
    timeout=1800,   # Timeout in 30 mins
    n_jobs=-1,     # Use all available cores
)

# Save the complete study to a file
study_filename = f"mlp_studies/mlp1_optuna_study_{time.strftime('%Y%m%d_%H%M%S')}.pkl"
joblib.dump(study_mlp1, study_filename)
print(f"Study saved to {study_filename}")

[I 2025-05-11 19:28:56,883] A new study created in memory with name: mlp1
[I 2025-05-11 19:29:33,943] Trial 2 pruned. 
[I 2025-05-11 19:29:35,094] Trial 4 pruned. 
[I 2025-05-11 19:30:12,989] Trial 9 pruned. 
[I 2025-05-11 19:30:19,263] Trial 0 pruned. 
[I 2025-05-11 19:30:26,272] Trial 3 pruned. 
[I 2025-05-11 19:30:51,725] Trial 10 pruned. 
[I 2025-05-11 19:31:12,894] Trial 12 pruned. 
[I 2025-05-11 19:31:14,282] Trial 5 pruned. 
[I 2025-05-11 19:31:42,235] Trial 11 pruned. 
[I 2025-05-11 19:31:50,066] Trial 15 pruned. 
[I 2025-05-11 19:31:55,731] Trial 8 pruned. 
[I 2025-05-11 19:32:16,075] Trial 7 finished with value: 1.05175964648907 and parameters: {'learning_rate': 0.0002173368767333051, 'weight_decay': 1.15688566273886e-06, 'n_hidden_1': 120, 'dropout_rate_1': 0.09801768322871252}. Best is trial 7 with value: 1.05175964648907.
[I 2025-05-11 19:32:24,064] Trial 16 pruned. 
[I 2025-05-11 19:32:43,295] Trial 1 finished with value: 1.0502006579668095 and parameters: {'learning_rate

Study saved to mlp_studies/mlp1_optuna_study_20250511_194041.pkl


In [7]:
# First, set up the ASHA pruner
asha_pruner = optuna.pruners.SuccessiveHalvingPruner(
    min_resource=30,        # Minimum number of steps before pruning
    reduction_factor=2,    # Reduction factor for successive halving
    min_early_stopping_rate=0
)

study_mlp2 = optuna.create_study(study_name='mlp2', direction="minimize", pruner=asha_pruner)
study_mlp2.optimize(
    lambda trial: objective_mlp(trial,
                            input_dim=dh.input_dim,
                            depth=2,
                            dataloaders=dh.get_nn_data('cv', batch_size=256),
                            max_epochs=150),
    n_trials=64,  # Number of trials to run
    timeout=3600,   # Timeout in 60 mins
    n_jobs=-1,     # Use all available cores
)

# Save the complete study to a file
study_filename = f"mlp_studies/mlp2_optuna_study_{time.strftime('%Y%m%d_%H%M%S')}.pkl"
joblib.dump(study_mlp2, study_filename)
print(f"Study saved to {study_filename}")

[I 2025-05-11 21:24:29,438] A new study created in memory with name: mlp2
[I 2025-05-11 21:26:09,861] Trial 0 pruned. 
[I 2025-05-11 21:26:13,741] Trial 1 pruned. 
[I 2025-05-11 21:26:52,187] Trial 2 pruned. 
[I 2025-05-11 21:27:50,251] Trial 9 pruned. 
[I 2025-05-11 21:28:03,993] Trial 3 pruned. 
[I 2025-05-11 21:29:15,136] Trial 8 pruned. 
[I 2025-05-11 21:30:49,688] Trial 13 pruned. 
[I 2025-05-11 21:31:02,251] Trial 11 pruned. 
[I 2025-05-11 21:31:04,336] Trial 4 pruned. 
[I 2025-05-11 21:31:33,661] Trial 10 pruned. 
[I 2025-05-11 21:31:34,477] Trial 12 pruned. 
[I 2025-05-11 21:31:46,753] Trial 5 finished with value: 1.059124463643783 and parameters: {'learning_rate': 0.03942954837252065, 'weight_decay': 0.0006509594859040266, 'n_hidden_1': 64, 'dropout_rate_1': 0.2347181201235215, 'n_hidden_2': 104, 'dropout_rate_2': 0.440154300020214}. Best is trial 5 with value: 1.059124463643783.
[I 2025-05-11 21:31:53,527] Trial 7 finished with value: 1.0479922172350762 and parameters: {'lear

Study saved to mlp_studies/mlp2_optuna_study_20250511_215925.pkl


In [8]:
# First, set up the ASHA pruner
asha_pruner = optuna.pruners.SuccessiveHalvingPruner(
    min_resource=30,        # Minimum number of steps before pruning
    reduction_factor=2,    # Reduction factor for successive halving
    min_early_stopping_rate=0
)

study_mlp3 = optuna.create_study(study_name='mlp3', direction="minimize", pruner=asha_pruner)
study_mlp3.optimize(
    lambda trial: objective_mlp(trial,
                            input_dim=dh.input_dim,
                            depth=3,
                            dataloaders=dh.get_nn_data('cv', batch_size=256),
                            max_epochs=200),
    n_trials=80,  # Number of trials to run
    timeout=7200,   # Timeout in 90 mins
    n_jobs=-1,     # Use all available cores
)

# Save the complete study to a file
study_filename = f"mlp_studies/mlp3_optuna_study_{time.strftime('%Y%m%d_%H%M%S')}.pkl"
joblib.dump(study_mlp3, study_filename)
print(f"Study saved to {study_filename}")

[I 2025-05-11 22:39:40,518] A new study created in memory with name: mlp3
[I 2025-05-11 22:41:40,311] Trial 5 pruned. 
[I 2025-05-11 22:41:43,935] Trial 7 pruned. 
[I 2025-05-11 22:42:01,915] Trial 3 pruned. 
[I 2025-05-11 22:42:09,143] Trial 2 pruned. 
[I 2025-05-11 22:42:55,919] Trial 0 pruned. 
[I 2025-05-11 22:43:47,547] Trial 9 pruned. 
[I 2025-05-11 22:44:12,488] Trial 10 pruned. 
[I 2025-05-11 22:44:26,910] Trial 1 pruned. 
[I 2025-05-11 22:44:28,323] Trial 11 pruned. 
[I 2025-05-11 22:45:52,747] Trial 13 pruned. 
[I 2025-05-11 22:46:12,276] Trial 12 pruned. 
[I 2025-05-11 22:46:22,926] Trial 14 pruned. 
[I 2025-05-11 22:46:49,385] Trial 16 pruned. 
[I 2025-05-11 22:49:10,723] Trial 20 pruned. 
[I 2025-05-11 22:49:55,818] Trial 17 pruned. 
[I 2025-05-11 22:50:38,937] Trial 19 pruned. 
[I 2025-05-11 22:52:40,569] Trial 18 pruned. 
[I 2025-05-11 22:52:41,039] Trial 6 finished with value: 1.0507598718007405 and parameters: {'learning_rate': 0.01296844650172018, 'weight_decay': 0.00

Study saved to mlp_studies/mlp3_optuna_study_20250511_234516.pkl
